In [1]:
from tensorflow.keras import models, layers
from ngdlm import models as ngdlmodels
from ngdlm import utils as ngdlutils
from tensorflow.keras.datasets import mnist
import numpy as np

# Train and test data.

In [2]:
(x_input_train, _), (x_input_test, y_output_test) = mnist.load_data()
x_input_train = x_input_train.astype("float32") / 255.0
x_input_test = x_input_test.astype("float32") / 255.0
print(x_input_train.shape)
print(x_input_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [ ]:
input_shape = (28, 28)
latent_dim = 128

encoder_input = layers.Input(shape=input_shape)
encoder_output = encoder_input
encoder_output = layers.Reshape(input_shape + (1,))(encoder_input)
encoder_output = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(encoder_output)
encoder_output = layers.MaxPooling2D((2, 2), padding="same")(encoder_output)
encoder_output = layers.Conv2D(8, (3, 3), activation="relu", padding="same")(encoder_output)
encoder_output = layers.MaxPooling2D((2, 2), padding="same")(encoder_output)
encoder_output = layers.Conv2D(8, (3, 3), activation="relu", padding="same")(encoder_output)
encoder_output = layers.MaxPooling2D((2, 2), padding="same")(encoder_output)
encoder_output = layers.Flatten()(encoder_output)
encoder = models.Model(encoder_input, encoder_output)

# Create the decoder.
decoder_input = layers.Input(shape=(latent_dim,))
decoder_output = decoder_input
#decoder_output = layers.Dense(128, activation="relu")(decoder_output)
decoder_output = layers.Reshape((4, 4, 8))(decoder_output)
decoder_output = layers.Conv2D(8, (3, 3), activation="relu", padding="same")(decoder_output)
decoder_output = layers.UpSampling2D((2, 2))(decoder_output)
decoder_output = layers.Conv2D(8, (3, 3), activation="relu", padding="same")(decoder_output)
decoder_output = layers.UpSampling2D((2, 2))(decoder_output)
decoder_output = layers.Conv2D(16, (3, 3), activation="relu")(decoder_output)
decoder_output = layers.UpSampling2D((2, 2))(decoder_output)
decoder_output = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(decoder_output)
decoder_output = layers.Reshape((28, 28))(decoder_output)
decoder = models.Model(decoder_input, decoder_output)

# Create the VAE.
vae = ngdlmodels.VAE(encoder, decoder, latent_dim=latent_dim)
vae.compile(optimizer="adam", loss="binary_crossentropy")
vae.summary()

# Train.
print("Train...")
history = vae.fit(
        x_input_train, x_input_train,
        epochs=100,
        batch_size=32,
        shuffle=True,
        validation_data=(x_input_test, x_input_test)
    )
    
# Evaluate.
print("Evaluate...")
loss = vae.evaluate(x_input_test, x_input_test)
print("Loss:", loss)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Encoder:
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 28, 28, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 16)   160         reshape[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 16)   0          

60000/60000 [==============================] - 65s 1ms/sample - loss: 0.1628 - val_loss: 0.1616
Epoch 20/100
60000/60000 [==============================] - 65s 1ms/sample - loss: 0.1625 - val_loss: 0.1609
Epoch 21/100
60000/60000 [==============================] - 65s 1ms/sample - loss: 0.1622 - val_loss: 0.1612
Epoch 22/100
60000/60000 [==============================] - 65s 1ms/sample - loss: 0.1621 - val_loss: 0.1610
Epoch 23/100
60000/60000 [==============================] - 65s 1ms/sample - loss: 0.1619 - val_loss: 0.1609
Epoch 24/100
60000/60000 [==============================] - 63s 1ms/sample - loss: 0.1617 - val_loss: 0.1618
Epoch 25/100
60000/60000 [==============================] - 64s 1ms/sample - loss: 0.1615 - val_loss: 0.1609
Epoch 26/100
60000/60000 [==============================] - 64s 1ms/sample - loss: 0.1615 - val_loss: 0.1604
Epoch 27/100
60000/60000 [==============================] - 64s 1ms/sample - loss: 0.1612 - val_loss: 0.1601
Epoch 28/100
60000/60000 [======

60000/60000 [==============================] - 66s 1ms/sample - loss: 0.1572 - val_loss: 0.1560
Epoch 95/100
60000/60000 [==============================] - 64s 1ms/sample - loss: 0.1572 - val_loss: 0.1562
Epoch 96/100
60000/60000 [==============================] - 65s 1ms/sample - loss: 0.1572 - val_loss: 0.1564
Epoch 97/100
60000/60000 [==============================] - 66s 1ms/sample - loss: 0.1572 - val_loss: 0.1563
Epoch 98/100
60000/60000 [==============================] - 67s 1ms/sample - loss: 0.1571 - val_loss: 0.1564
Epoch 99/100
30048/60000 [==============>...............] - ETA: 31s - loss: 0.1570

# Visualizing.

In [ ]:
print("Rendering history...")
ngdlutils.render_history(history)

print("Rendering reconstructions...")
ngdlutils.render_image_reconstructions(vae, x_input_train[0:10])

print("Rendering latent-space...")
ngdlutils.render_image_latent_space(vae.decoder, 10)

print("Rendering encodings...")
ngdlutils.render_encodings(vae.encoder, x_input_test, y_output_test)